<a href="https://colab.research.google.com/github/dercodeKoenig/hft_dqn/blob/main/hft_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r *
!git clone https://github.com/dercodeKoenig/hft_dqn

Cloning into 'hft_dqn'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 85 (delta 0), reused 2 (delta 0), pack-reused 80 (from 1)
Receiving objects: 100% (85/85), 60.97 MiB | 8.63 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Updating files: 100% (18/18), done.


In [2]:
%cd hft_dqn

/content/hft_dqn


In [3]:
!git pull

Already up to date.


In [4]:
from datetime import datetime, timedelta
import time
from collections import deque
import numpy as np
import copy
import tensorflow as tf
import random
from save_and_load import *
from Candle import Candle
from MultiTimeframeCandleManager import MultiTimeframeCandleManager

candles = obj_load("NQ_1")
len(candles)

loading NQ_1


850000

In [5]:
gamma = 0.95
memory_len = 100000
batch_size = 128
e = 2


ep_len = 1000

m1 = np.eye(2, dtype="float32")
num_model_inputs = 7

In [7]:
chart_m15 = tf.keras.layers.Input(shape = (60,4))
chart_m5 = tf.keras.layers.Input(shape = (60,4))
chart_m1 = tf.keras.layers.Input(shape = (60,4))

pdas = tf.keras.layers.Input(shape = (19,))

current_position = tf.keras.layers.Input(shape = (1,))
scaled_open_profit = tf.keras.layers.Input(shape = (1,))

minutes = tf.keras.layers.Input(shape = (1,))
minutes_embed = tf.keras.layers.Embedding(input_dim=60*24, output_dim=8)(minutes)
minutes_embed_flat = tf.keras.layers.Flatten()(minutes_embed)


r15 = tf.keras.layers.GRU(64, return_sequences=True)(chart_m15)
r5 = tf.keras.layers.GRU(64, return_sequences=True)(chart_m5)
r1 = tf.keras.layers.GRU(64, return_sequences=True)(chart_m1)

r15 = tf.keras.layers.GRU(64, return_sequences=False)(r15)
r5 = tf.keras.layers.GRU(64, return_sequences=False)(r5)
r1 = tf.keras.layers.GRU(64, return_sequences=False)(r1)


f15 = tf.keras.layers.Flatten()(chart_m15)
f5 = tf.keras.layers.Flatten()(chart_m5)
f1 = tf.keras.layers.Flatten()(chart_m1)

c = tf.keras.layers.Concatenate()([f15, f5, f1, r15,r5,r1, pdas, minutes_embed_flat, current_position, scaled_open_profit])

lrelu = tf.keras.layers.LeakyReLU(0.1)
d = tf.keras.layers.Dense(4096)(c)
d = lrelu(d)
d = tf.keras.layers.Dense(2048)(d)
d = lrelu(d)
d = tf.keras.layers.Dense(1024)(d)
d = lrelu(d)

value = tf.keras.layers.Dense(1, activation="linear")(d)
advantage = tf.keras.layers.Dense(2, activation="linear")(d)

q_values = tf.keras.layers.Lambda(
    lambda inputs: inputs[0] + (inputs[1] - tf.reduce_mean(inputs[1], axis=1, keepdims=True))
)([value, advantage])


model = tf.keras.Model(inputs = [chart_m15, chart_m5, chart_m1, pdas, minutes, current_position, scaled_open_profit], outputs = q_values)
target_model = tf.keras.Model(inputs = [chart_m15, chart_m5, chart_m1, pdas, minutes, current_position, scaled_open_profit], outputs = q_values)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 60, 4)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_8             │ (None, 60, 4)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_9             │ (None, 60, 4)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_13            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_6 (GRU)               │ (None, 60, 64)         │         13,440 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_7 (GRU)               │ (None, 60, 64)         │         13,440 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_8 (GRU)               │ (None, 60, 64)         │         13,440 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 1, 8)           │         11,520 │ input_layer_13[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_5 (Flatten)       │ (None, 240)            │              0 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_6 (Flatten)       │ (None, 240)            │              0 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_7 (Flatten)       │ (None, 240)            │              0 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_9 (GRU)               │ (None, 64)             │         24,960 │ gru_6[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_10 (GRU)              │ (None, 64)             │         24,960 │ gru_7[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_11 (GRU)              │ (None, 64)             │         24,960 │ gru_8[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_10            │ (None, 19)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_4 (Flatten)       │ (None, 8)              │              0 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_11            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)         

 Total params: 14,477,059 (55.23 MB)

 Trainable params: 14,477,059 (55.23 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001)

In [9]:
try:
    model.load_weights("model.weights.h5")
    target_model.load_weights("model.weights.h5")
except:
    print("unable to load weights")

unable to load weights


In [10]:

def relative (value, center, r):
        return (value - center) / r

def ret_to_scaled_inputs(ret):

    midnight_open, midnight_opening_range_high,midnight_opening_range_low, pdas, current_close, current_time, charts = ret

    center = (midnight_opening_range_high + midnight_opening_range_low) / 2
    r = (midnight_opening_range_high - midnight_opening_range_low) / 2

    pda_rel = []
    pda_rel.append(relative(midnight_open, center, r))
    for pda in pdas:
        pda_rel.append(relative(pda, center, r))
    pda_np = np.array(pda_rel)

    current_minutes = current_time.hour * 60 + current_time.minute

    charts_array = []
    for candlesticks in charts:
        charts_array.append([])
        for candle in candlesticks:
            o = relative(candle.o, center, r)
            h = relative(candle.h, center, r)
            l = relative(candle.l, center, r)
            c = relative(candle.c, center, r)
            charts_array[-1].append([o,h,l,c])

    m15_np = np.array(charts_array[0])
    m5_np = np.array(charts_array[1])
    m1_np = np.array(charts_array[2])

    return [m15_np, m5_np, m1_np, pda_np, current_minutes]

In [11]:
m = MultiTimeframeCandleManager()

current_position = 0
entry_price = 0

equity = 0
open_profit = 0
equity_L = [0]

cmm = 0.5

last_close = 0
last_state = None
last_action = 0

index = 0

def step():

    global index, last_close, last_state, last_action, current_position, entry_price, equity


    sarts = None
    while  sarts == None:

        ret = m.push_m1_candle(candles[index])
        midnight_open, midnight_opening_range_high,midnight_opening_range_low, pdas, current_close, current_time, charts = ret
        center = (midnight_opening_range_high + midnight_opening_range_low) / 2
        r = (midnight_opening_range_high - midnight_opening_range_low) / 2


        if(len(m.m15_candles) == 60):

            open_profit = (current_close - entry_price) * current_position
            scaled_open_profit = open_profit / r

            state = ret_to_scaled_inputs(ret) + [current_position, scaled_open_profit]
            m15_np, m5_np, m1_np, pda_np, current_minutes, pos, open_profit = state

            if(last_state != None):
                diff = (equity+open_profit) - equity_L[-1]
                equity_L.append(equity+open_profit)
                reward =  (diff) / r
                terminal = 0
                if(index+1 == len(candles)):
                    terminal = 1

                sarts = last_state, last_action, reward, terminal, state


            if(random.randint(0,100) > e):
                output = model([
                    tf.expand_dims(m15_np, 0),
                    tf.expand_dims(m5_np, 0),
                    tf.expand_dims(m1_np, 0),
                    tf.expand_dims(pda_np, 0),
                    tf.expand_dims(current_minutes, 0),
                    tf.expand_dims(pos, 0),
                    tf.expand_dims(open_profit, 0),
                ])

                last_action = np.argmax(output)
            else:
                last_action = random.randint(0,1)

            last_close = current_close
            last_state = state


            if(last_action == 0 and current_position != -1):
                equity += open_profit
                #print("short at", current_close, " last position return:", open_profit, " equity:", equity)
                current_position = -1
                entry_price = current_close
                equity -= cmm

            if(last_action == 1 and current_position != 1):
                equity += open_profit
                #print("long at", current_close, " last position return:", open_profit, " equity:", equity)
                current_position = 1
                entry_price = current_close
                equity -= cmm



        index += 1
        if(index == len(candles)):
            index = 0
            current_position = 0
            entry_price = 0
            last_close = 0
            last_state = None
            last_action = 0
            print("env reset")

    return sarts

In [12]:
@tf.function(reduce_retracing=True)
def get_target_q(next_states, rewards, terminals):
            estimated_q_values_next = target_model(next_states)
            q_batch = tf.math.reduce_max(estimated_q_values_next, axis=1)
            target_q_values = q_batch * gamma * (1-terminals) + rewards
            return target_q_values

@tf.function(reduce_retracing=True)
def tstep(states, masks, rewards, terminals, next_states):

    target_q_values = get_target_q(next_states, rewards, terminals)

    with tf.GradientTape() as t:
        model_return = model(states, training=True)
        mask_return = model_return * masks
        estimated_q_values = tf.math.reduce_sum(mask_return, axis=1)
        #print(estimated_q_values, mask_return, model_return, masks)
        loss_e = tf.math.square(target_q_values - estimated_q_values)
        loss = tf.reduce_mean(loss_e)


    gradient = t.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

    return loss, tf.reduce_mean(estimated_q_values)

In [13]:
def run():
    sarts = step()
    sarts_memory.append(sarts)

    sarts_sample = random.sample(sarts_memory, min(batch_size, len(sarts_memory)))

    states = [x[0] for x in sarts_sample]
    actions = [x[1] for x in sarts_sample]
    rewards = np.array([x[2] for x in sarts_sample], dtype="float32")
    terminals = np.array([x[3] for x in sarts_sample], dtype="float32")
    next_states = [x[4] for x in sarts_sample]

    next_states_array = []
    for i in range(num_model_inputs):
        next_states_array.append(np.array([x[i] for x in next_states], dtype = "float32"))


    states_array = []
    for i in range(num_model_inputs):
        states_array.append(np.array([x[i] for x in states], dtype = "float32"))


    masks = np.array(m1[actions], dtype="float32")

    loss, q = tstep(states_array, masks, rewards, terminals, next_states_array)

    return loss, q, sarts[2], sarts[1]

In [14]:
sarts_memory = deque(maxlen = memory_len)

loss_mean = []
q_mean = []
rewards = []

try:
    loss_mean = obj_load("loss")
    q_mean = obj_load("q")
    rewards = obj_load("rewards")
except:
    print("unable to load data")

loading loss
loading q
loading rewards


In [ ]:


t0 = time.time()
safe_after_eps = 10
eps_counter=0

while True:
    eps_counter+=1
    try:
        loss = []
        q = []
        rewards_tmp = []
        actions = []
        progbar = tf.keras.utils.Progbar(ep_len)
        for i in range(ep_len):
            c_loss, c_q, c_rewards, c_action = run()
            loss.append(c_loss)
            q.append(c_q)
            #rewards_tmp.append(c_rewards)
            rewards.append(c_rewards)
            actions.append(c_action)

            progbar.update(i+1, values = [("loss", c_loss), ("qv", c_q), ("reward", c_rewards), ("avg_action", c_action)])

        loss_mean.append(np.mean(loss))
        q_mean.append(np.mean(q))
        #rewards.append(np.mean(rewards_tmp))


        #progbar.update(ep_len, values = [("loss", np.mean(loss)), ("qv", np.mean(q)), ("reward", np.mean(rewards)), ("avg_action", np.mean(actions))])

        target_model.set_weights(model.get_weights())

        if(eps_counter >= safe_after_eps):
            eps_counter = 0
            model.save_weights("model.weights.h5")
            obj_save(loss_mean, "loss")
            obj_save(q_mean, "q")
            obj_save(rewards, "rewards")
            print("saved progress")

    except    KeyboardInterrupt:
        print("")
        print("exit")
        model.save_weights("model.weights.h5")
        obj_save(loss_mean, "loss")
        obj_save(q_mean, "q")
        obj_save(rewards, "rewards")
        break


 630/1000 ━━━━━━━━━━━━━━━━━━━━ 42s 114ms/step - loss: 14.3108 - qv: 25.3877 - reward: -0.0291 - avg_action: 0.4746

In [ ]:
#import matplotlib.pyplot as plt
#plt.plot(equity_L)
#equity